In [1]:
# Cell: imports
import pandas as pd
import numpy as np
from pathlib import Path
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error

# display settings
pd.set_option('display.max_columns', 200)


In [2]:
# Cell: load data
DATA_DIR = Path(".")  # change if your files are elsewhere
patients = pd.read_csv(DATA_DIR / "patients.csv")
services = pd.read_csv(DATA_DIR / "services_weekly.csv")
staff = pd.read_csv(DATA_DIR / "staff.csv")
staff_schedule = pd.read_csv(DATA_DIR / "staff_schedule.csv")

print("patients columns:", patients.columns.tolist())
print("services columns:", services.columns.tolist())
print("staff columns:", staff.columns.tolist())
print("staff_schedule columns:", staff_schedule.columns.tolist())

# show top rows
display(patients.head())
display(services.head())
display(staff.head())
display(staff_schedule.head())


patients columns: ['patient_id', 'name', 'age', 'arrival_date', 'departure_date', 'service', 'satisfaction']
services columns: ['week', 'month', 'service', 'available_beds', 'patients_request', 'patients_admitted', 'patients_refused', 'patient_satisfaction', 'staff_morale', 'event']
staff columns: ['staff_id', 'staff_name', 'role', 'service']
staff_schedule columns: ['week', 'staff_id', 'staff_name', 'role', 'service', 'present']


,patient_id,name,age,arrival_date,departure_date,service,satisfaction
0,PAT-09484753,Richard Rodriguez,24,2025-03-16,2025-03-22,surgery,61
1,PAT-f0644084,Shannon Walker,6,2025-12-13,2025-12-14,surgery,83
2,PAT-ac6162e4,Julia Torres,24,2025-06-29,2025-07-05,general_medicine,83
3,PAT-3dda2bb5,Crystal Johnson,32,2025-10-12,2025-10-23,emergency,81
4,PAT-08591375,Garrett Lin,25,2025-02-18,2025-02-25,ICU,76


,week,month,service,available_beds,patients_request,patients_admitted,patients_refused,patient_satisfaction,staff_morale,event
0,1,1,emergency,32,76,32,44,67,70,none
1,1,1,surgery,45,130,45,85,83,78,flu
2,1,1,general_medicine,37,201,37,164,97,43,flu
3,1,1,ICU,22,31,22,9,84,91,flu
4,2,1,emergency,28,169,28,141,75,64,none


,staff_id,staff_name,role,service
0,STF-5ca26577,Allison Hill,doctor,emergency
1,STF-02ae59ca,Noah Rhodes,doctor,emergency
2,STF-d8006e7c,Angie Henderson,doctor,emergency
3,STF-212d8b31,Daniel Wagner,doctor,emergency
4,STF-107a58e4,Cristian Santos,doctor,emergency


,week,staff_id,staff_name,role,service,present
0,1,STF-b77cdc60,Allison Hill,doctor,emergency,1
1,2,STF-b77cdc60,Allison Hill,doctor,emergency,1
2,3,STF-b77cdc60,Allison Hill,doctor,emergency,0
3,4,STF-b77cdc60,Allison Hill,doctor,emergency,1
4,5,STF-b77cdc60,Allison Hill,doctor,emergency,1


In [3]:
# Convert date columns and engineer Week and LengthOfStay
# (The uploaded patient file used arrival_date/departure_date and satisfaction)
patients['arrival_date'] = pd.to_datetime(patients['arrival_date'], errors='coerce')
patients['departure_date'] = pd.to_datetime(patients['departure_date'], errors='coerce')

# ISO week number (1-53)
patients['Week'] = patients['arrival_date'].dt.isocalendar().week.astype(int)

# LengthOfStay in days
patients['LengthOfStay'] = (patients['departure_date'] - patients['arrival_date']).dt.days.fillna(0).astype(int)

# Quick check
patients[['arrival_date','departure_date','Week','LengthOfStay']].head()


,arrival_date,departure_date,Week,LengthOfStay
0,2025-03-16,2025-03-22,11,6
1,2025-12-13,2025-12-14,50,1
2,2025-06-29,2025-07-05,26,6
3,2025-10-12,2025-10-23,41,11
4,2025-02-18,2025-02-25,8,7


In [5]:
# Normalize service columns to names we will use in the model
services = services.rename(columns={
    'week': 'Week',
    'service': 'Service',
    'available_beds': 'BedsAvailable',
    'patients_request': 'PatientDemand',
    'patient_satisfaction': 'ServiceSatisfaction'
})

# Ensure Week numeric
services['Week'] = services['Week'].astype(int)

# Quick preview
services.head()


,Week,month,Service,BedsAvailable,PatientDemand,patients_admitted,patients_refused,ServiceSatisfaction,staff_morale,event
0,1,1,emergency,32,76,32,44,67,70,none
1,1,1,surgery,45,130,45,85,83,78,flu
2,1,1,general_medicine,37,201,37,164,97,43,flu
3,1,1,ICU,22,31,22,9,84,91,flu
4,2,1,emergency,28,169,28,141,75,64,none


In [6]:
# Merge patients with services on Week and Service
# patient side uses 'service' column name; ensure it's consistent
patients['Service'] = patients['service']
merged = pd.merge(patients, services, how='left', left_on=['Week', 'Service'], right_on=['Week', 'Service'])

# Show shape and missing merge rows
print("Merged shape:", merged.shape)
print("Missing BedsAvailable after merge:", merged['BedsAvailable'].isna().sum())
display(merged.head())


Merged shape: (1000, 18)
Missing BedsAvailable after merge: 0


,patient_id,name,age,arrival_date,departure_date,service,satisfaction,Week,LengthOfStay,Service,month,BedsAvailable,PatientDemand,patients_admitted,patients_refused,ServiceSatisfaction,staff_morale,event
0,PAT-09484753,Richard Rodriguez,24,2025-03-16,2025-03-22,surgery,61,11,6,surgery,3,48,48,48,0,82,80,none
1,PAT-f0644084,Shannon Walker,6,2025-12-13,2025-12-14,surgery,83,50,1,surgery,12,59,65,59,6,95,85,none
2,PAT-ac6162e4,Julia Torres,24,2025-06-29,2025-07-05,general_medicine,83,26,6,general_medicine,7,43,72,43,29,80,85,none
3,PAT-3dda2bb5,Crystal Johnson,32,2025-10-12,2025-10-23,emergency,81,41,11,emergency,11,27,92,27,65,69,88,none
4,PAT-08591375,Garrett Lin,25,2025-02-18,2025-02-25,ICU,76,8,7,ICU,2,12,10,10,0,76,98,none


In [7]:
# Target in this dataset is 'satisfaction' in patients.csv
TARGET = 'satisfaction'
if TARGET not in merged.columns:
    raise RuntimeError(f"Target column '{TARGET}' not found.")

# Drop rows missing the target
merged = merged.dropna(subset=[TARGET]).copy()

# Fill missing numeric service-level features with sensible defaults (0 or median as appropriate)
merged['BedsAvailable'] = merged['BedsAvailable'].fillna(0).astype(int)
merged['PatientDemand'] = merged['PatientDemand'].fillna(0).astype(int)

# NOTE: We'll generate StaffAssigned later using ActualStaffPresent. For now create placeholder 0 if missing
merged['StaffAssigned'] = merged.get('StaffAssigned', np.nan)

# Feature set chosen per assignment
feature_cols = ['LengthOfStay','Service','BedsAvailable','PatientDemand','StaffAssigned']
display(merged[feature_cols + [TARGET]].head())


,LengthOfStay,Service,BedsAvailable,PatientDemand,StaffAssigned,satisfaction
0,6,surgery,48,48,NaN,61
1,1,surgery,59,65,NaN,83
2,6,general_medicine,43,72,NaN,83
3,11,emergency,27,92,NaN,81
4,7,ICU,12,10,NaN,76


In [9]:
# Preprocessor: OneHot encode Service, scale numerics
numeric_features = ['LengthOfStay','BedsAvailable','PatientDemand','StaffAssigned']
categorical_features = ['Service']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])


In [10]:
# For initial model we'll drop rows where StaffAssigned is missing (we'll build ActualStaffPresent in bonus)
data_initial = merged.dropna(subset=['StaffAssigned']).copy() if merged['StaffAssigned'].notna().any() else None

if data_initial is None:
    print("No pre-existing StaffAssigned values found. Skipping initial-only models. Proceeding to bonus staffing to compute ActualStaffPresent.")
else:
    X = data_initial[feature_cols].copy()
    y = data_initial[TARGET].astype(float).copy()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    baseline_pipe = Pipeline([('pre', preprocessor), ('model', LinearRegression())])
    baseline_pipe.fit(X_train, y_train)
    y_pred_base = baseline_pipe.predict(X_test)
    print("Baseline R2:", r2_score(y_test, y_pred_base), "MAE:", mean_absolute_error(y_test, y_pred_base))

    rf_pipe = Pipeline([('pre', preprocessor), ('model', RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1))])
    rf_pipe.fit(X_train, y_train)
    y_pred_rf = rf_pipe.predict(X_test)
    print("RF R2:", r2_score(y_test, y_pred_rf), "MAE:", mean_absolute_error(y_test, y_pred_rf))


No pre-existing StaffAssigned values found. Skipping initial-only models. Proceeding to bonus staffing to compute ActualStaffPresent.


In [14]:
# Build ActualStaffPresent from staff + staff_schedule
# staff_schedule likely has columns: week, staff_id, staff_name, role, service, present (1/0)
staff_merged = staff_schedule.merge(staff, on='staff_id', how='left')

# Determine the correct service column from the merge (schedule's service e.g., service_x)
if 'service_x' in staff_merged.columns:
    staff_merged['Service'] = staff_merged['service_x']
elif 'service' in staff_merged.columns:
    staff_merged['Service'] = staff_merged['service']
else:
    raise RuntimeError("No service column in staff_schedule/staff after merge.")

# Ensure present is numeric binary
staff_merged['present'] = staff_merged['present'].astype(int)

# Aggregate: count distinct staff present per (week, Service)
actual = (staff_merged[staff_merged['present'] == 1]
          .groupby(['week', 'Service'], as_index=False)
          .agg(ActualStaffPresent=('staff_id', 'nunique')))
actual = actual.rename(columns={'week':'Week'})

# Show
display(actual.head())


,Week,Service,ActualStaffPresent
0,1,ICU,31
1,1,emergency,35
2,1,general_medicine,27
3,1,surgery,23
4,2,ICU,30


In [15]:
# Merge actual staffing into services on Week & Service
services2 = services.merge(actual, how='left', left_on=['Week','Service'], right_on=['Week','Service'])
services2['ActualStaffPresent'] = services2['ActualStaffPresent'].fillna(0).astype(int)

# Now merge this into patient-level merged dataset
merged2 = patients.merge(services2, how='left', left_on=['Week','service'], right_on=['Week','Service'])

# Set features: use ActualStaffPresent as StaffAssigned
merged2['Service'] = merged2['service']
merged2['BedsAvailable'] = merged2['BedsAvailable'].fillna(0).astype(int)
merged2['PatientDemand'] = merged2['PatientDemand'].fillna(0).astype(int)
merged2['StaffAssigned'] = merged2['ActualStaffPresent'].fillna(0).astype(int)

# Ensure target present and drop nulls
merged2 = merged2.dropna(subset=[TARGET])

# Check
display(merged2[['Week','Service','BedsAvailable','PatientDemand','ActualStaffPresent','StaffAssigned']].head())


,Week,Service,BedsAvailable,PatientDemand,ActualStaffPresent,StaffAssigned
0,11,surgery,48,48,23,23
1,50,surgery,59,65,21,21
2,26,general_medicine,43,72,25,25
3,41,emergency,27,92,37,37
4,8,ICU,12,10,28,28


In [17]:
# Train-test split on merged2
feature_cols = ['LengthOfStay','Service','BedsAvailable','PatientDemand','StaffAssigned']
X = merged2[feature_cols].copy()
y = merged2[TARGET].astype(float).copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Recreate preprocessor (ensure numeric features include StaffAssigned)
numeric_features = ['LengthOfStay','BedsAvailable','PatientDemand','StaffAssigned']
categorical_features = ['Service']

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# Baseline linear
baseline_pipe = Pipeline([('pre', preprocessor), ('model', LinearRegression())])
baseline_pipe.fit(X_train, y_train)
y_pred_base = baseline_pipe.predict(X_test)
baseline_r2 = r2_score(y_test, y_pred_base)
baseline_mae = mean_absolute_error(y_test, y_pred_base)

# Random forest
rf_pipe = Pipeline([('pre', preprocessor), ('model', RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1))])
rf_pipe.fit(X_train, y_train)
y_pred_rf = rf_pipe.predict(X_test)
rf_r2 = r2_score(y_test, y_pred_rf)
rf_mae = mean_absolute_error(y_test, y_pred_rf)

# summary
print("Baseline LinearRegression: R2 = {:.4f}, MAE = {:.4f}".format(baseline_r2, baseline_mae))
print("RandomForest:            R2 = {:.4f}, MAE = {:.4f}".format(rf_r2, rf_mae))

# choose best by R2
best_pipe = rf_pipe if rf_r2 >= baseline_r2 else baseline_pipe
best_name = "RandomForest" if best_pipe is rf_pipe else "LinearRegression"
print("Chosen model:", best_name)



Baseline LinearRegression: R2 = -0.0042, MAE = 10.3955
RandomForest:            R2 = -0.2965, MAE = 11.6085
Chosen model: LinearRegression


In [18]:
# If RandomForest chosen, extract feature importances
if best_name == "RandomForest":
    # create feature names: numeric features + OHE categories
    cat_feature_names = rf_pipe.named_steps['pre'].named_transformers_['cat'].get_feature_names_out(categorical_features)
    feature_names = numeric_features + list(cat_feature_names)
    importances = rf_pipe.named_steps['model'].feature_importances_
    fi = pd.DataFrame({'feature': feature_names, 'importance': importances}).sort_values('importance', ascending=False)
    display(fi.head(20))

    # plot top 10
    top = fi.head(10).iloc[::-1]
    plt.figure(figsize=(8,5))
    plt.barh(top['feature'], top['importance'])
    plt.title("Top 10 Feature Importances (RandomForest)")
    plt.xlabel("Importance")
    plt.show()
else:
    print("RandomForest not chosen; cannot extract importances. Consider training RF.")
    


RandomForest not chosen; cannot extract importances. Consider training RF.


In [19]:
# Save best pipeline (preprocessor + model) to rf_bonus_pipeline.joblib in working dir
out_path = Path("rf_bonus_pipeline.joblib")
joblib.dump(best_pipe, out_path)
print("Saved pipeline to:", out_path.resolve())


Saved pipeline to: C:\Users\user\Desktop\Patientsatisfctrymodel\rf_bonus_pipeline.joblib


In [20]:
# Quick check: use pipeline to predict a sample patient
sample = pd.DataFrame([{
    'LengthOfStay': 3,
    'Service': 'emergency',
    'BedsAvailable': 30,
    'PatientDemand': 100,
    'StaffAssigned': 8
}])
pred = best_pipe.predict(sample)[0]
print("Predicted satisfaction:", pred)


Predicted satisfaction: 78.62149057610054
